In [1]:
import pandas as pd
import numpy as np
import xgboost as xgb
from sklearn.model_selection import train_test_split
from sklearn.model_selection import cross_val_score

In [2]:
dest = '/content/drive/MyDrive/HSE Final Project/to_train'
name = 'submit_6_abandoned.csv'

df = pd.read_csv(dest + '/' + name)
df.head()

,shop_id,item_id,month,year,date_block_num,item_cnt,item_price,prev_value,prev_value_year,stock_prev,stock_prev_year,shop_city,shop_category,item_category_id,digital,category_type,category_subtype,item_cnt_lag_2,item_cnt_lag_3,item_cnt_lag_4,item_cnt_lag_5
0,0,19,1,2013,0,0,28.0,-1,-1,-1.0,-1.0,29,4,40,False,7,4,0,0,0,0
1,0,27,1,2013,0,0,2499.0,-1,-1,-1.0,-1.0,29,4,19,False,5,9,0,0,0,0
2,0,28,1,2013,0,0,549.0,-1,-1,-1.0,-1.0,29,4,30,False,5,49,0,0,0,0
3,0,29,1,2013,0,0,2499.0,-1,-1,-1.0,-1.0,29,4,23,False,5,18,0,0,0,0
4,0,32,1,2013,0,6,349.0,-1,-1,-1.0,-1.0,29,4,40,False,7,4,0,0,0,0


In [3]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10913804 entries, 0 to 10913803
Data columns (total 21 columns):
 #   Column            Dtype  
---  ------            -----  
 0   shop_id           int64  
 1   item_id           int64  
 2   month             int64  
 3   year              int64  
 4   date_block_num    int64  
 5   item_cnt          int64  
 6   item_price        float64
 7   prev_value        int64  
 8   prev_value_year   int64  
 9   stock_prev        float64
 10  stock_prev_year   float64
 11  shop_city         int64  
 12  shop_category     int64  
 13  item_category_id  int64  
 14  digital           bool   
 15  category_type     int64  
 16  category_subtype  int64  
 17  item_cnt_lag_2    int64  
 18  item_cnt_lag_3    int64  
 19  item_cnt_lag_4    int64  
 20  item_cnt_lag_5    int64  
dtypes: bool(1), float64(3), int64(17)
memory usage: 1.6 GB


In [4]:
X = df.drop('item_cnt', axis=1)
y = df.loc[:, 'item_cnt']

Due to the fact that we predict future, it makes no sense to shuffle the data

In [5]:
X_train, X_test, y_train, y_test = train_test_split(X, y,
                                                    test_size=0.3,
                                                    shuffle=False,
                                                    random_state=7)

In [6]:
learning_rate = 0.1
max_depth = 7
random_state = 7
verbosity = 2

In [7]:
xgbr = xgb.XGBRegressor(verbosity=verbosity,
                        eta=learning_rate,
                        max_depth=max_depth,
                        random_state=random_state)

In [8]:
xgbr.fit(X_train, y_train)

[12:50:38] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[12:50:38] WARNING: /workspace/src/learner.cc:686: Tree method is automatically selected to be 'approx' for faster speed. To use old behavior (exact greedy algorithm on single machine), set tree_method to 'exact'.
[12:51:18] INFO: /workspace/src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 226 extra nodes, 0 pruned nodes, max_depth=7
[12:51:52] INFO: /workspace/src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 228 extra nodes, 0 pruned nodes, max_depth=7
[12:52:26] INFO: /workspace/src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 220 extra nodes, 0 pruned nodes, max_depth=7
[12:53:01] INFO: /workspace/src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 220 extra nodes, 0 pruned nodes, max_depth=7
[12:53:34] INFO: /workspace/src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 236 extra nodes, 0 pruned nodes, max_depth=7
[12:54:06

XGBRegressor(eta=0.1, max_depth=7, random_state=7, verbosity=2)

In [9]:
from sklearn.metrics import mean_squared_error

y_pred = xgbr.predict(X_test)
mse = mean_squared_error(y_test, y_pred)

print('MSE: %.2f' % mse)
print('RMSE: %.2f' % (mse ** (0.5)))

MSE: 10.48
RMSE: 3.24


In [10]:
# from sklearn.metrics import mean_squared_log_error
# msle = mean_squared_log_error(y_test, y_pred)
# print('RMSLE: %.2f' % (msle ** (0.5)))

In [16]:
dest = '/content/drive/MyDrive/HSE Final Project/to_test'
name = 'submit_6_test.csv'

test_df = pd.read_csv(dest + '/' + name)
test_df.head()

,shop_id,item_id,year,month,date_block_num,item_price,prev_value,prev_value_year,stock_prev,stock_prev_year,shop_city,shop_category,item_category_id,digital,category_type,category_subtype,item_cnt_lag_2,item_cnt_lag_3,item_cnt_lag_4,item_cnt_lag_5
0,5,5037,2015,11,34,2599.0,0,1,0.0,2599.0,3,3,19,False,5,9,1,3,1,1
1,5,5320,2015,11,34,-1.0,-1,-1,1.0,1.0,3,3,55,False,9,2,0,0,0,0
2,5,5233,2015,11,34,1199.0,1,-1,1199.0,-1.0,3,3,19,False,5,9,3,1,0,2
3,5,5232,2015,11,34,1199.0,0,-1,0.0,-1.0,3,3,23,False,5,18,0,1,0,0
4,5,5268,2015,11,34,-1.0,-1,-1,1.0,1.0,3,3,20,False,5,10,0,0,0,0


In [17]:
test_df[['year', 'month']] = test_df[['month', 'year']]
test_df.head()

,shop_id,item_id,year,month,date_block_num,item_price,prev_value,prev_value_year,stock_prev,stock_prev_year,shop_city,shop_category,item_category_id,digital,category_type,category_subtype,item_cnt_lag_2,item_cnt_lag_3,item_cnt_lag_4,item_cnt_lag_5
0,5,5037,11,2015,34,2599.0,0,1,0.0,2599.0,3,3,19,False,5,9,1,3,1,1
1,5,5320,11,2015,34,-1.0,-1,-1,1.0,1.0,3,3,55,False,9,2,0,0,0,0
2,5,5233,11,2015,34,1199.0,1,-1,1199.0,-1.0,3,3,19,False,5,9,3,1,0,2
3,5,5232,11,2015,34,1199.0,0,-1,0.0,-1.0,3,3,23,False,5,18,0,1,0,0
4,5,5268,11,2015,34,-1.0,-1,-1,1.0,1.0,3,3,20,False,5,10,0,0,0,0


In [21]:
test_df = test_df.rename(columns={'year': 'month',
                                  'month': 'year'})

In [22]:
test_df.head()

,shop_id,item_id,month,year,date_block_num,item_price,prev_value,prev_value_year,stock_prev,stock_prev_year,shop_city,shop_category,item_category_id,digital,category_type,category_subtype,item_cnt_lag_2,item_cnt_lag_3,item_cnt_lag_4,item_cnt_lag_5
0,5,5037,11,2015,34,2599.0,0,1,0.0,2599.0,3,3,19,False,5,9,1,3,1,1
1,5,5320,11,2015,34,-1.0,-1,-1,1.0,1.0,3,3,55,False,9,2,0,0,0,0
2,5,5233,11,2015,34,1199.0,1,-1,1199.0,-1.0,3,3,19,False,5,9,3,1,0,2
3,5,5232,11,2015,34,1199.0,0,-1,0.0,-1.0,3,3,23,False,5,18,0,1,0,0
4,5,5268,11,2015,34,-1.0,-1,-1,1.0,1.0,3,3,20,False,5,10,0,0,0,0


In [23]:
test_pred = xgbr.predict(test_df)
print(test_pred)

[ 0.43505192 -0.01489866  0.9071518  ...  0.05030483  0.04486042
  0.03644663]


In [24]:
name_sample = 'sample_submission.csv'

sample = pd.read_csv(dest + '/' + name_sample)
sample.head()

,ID,item_cnt_month
0,0,0.5
1,1,0.5
2,2,0.5
3,3,0.5
4,4,0.5


In [25]:
sample.item_cnt_month = np.round(test_pred)
sample.head()

,ID,item_cnt_month
0,0,0.0
1,1,-0.0
2,2,1.0
3,3,0.0
4,4,1.0


In [27]:
dest = '/content/drive/MyDrive/HSE Final Project/submissions/'
sample.to_csv(dest + 'submission #10 (some lags).csv', index=False)